# 構造化データセットSFT（Unsloth / Colab T4）標準学習コード：実行ガイド

本ノートブックは、**構造化出力を測るベンチマークスコア向上**を目的として、  
小型LLM（Qwen3-4B Instruct-2507）に対して **SFT（Supervised Fine-Tuning）** を行う標準コードです。

学習は **Unsloth + QLoRA（4bit）** を利用し、**Colab 無料版（T4）**で動作するようにメモリ最適化されています。



## 1. このコードが行うこと（概要）



このコードは大きく3段階で構成されています。

1. **環境固定（依存パッケージのバージョン固定）**  
   Colabの環境変化による不具合を避けるため、numpy/transformers/trl/unsloth等を特定バージョンで揃えます。

2. **SFT（教師あり微調整）の実行**  
   Hugging Face Hub 上の学習データセットを読み込み、ベースモデルに LoRA アダプタを差し込み、学習します。  
   学習の損失（loss）は **assistant 出力部分だけ**にかかる設計です（structured output を学習させやすい）。

3. **LoRAアダプタのHugging Faceへのアップロード**  
   学習で得られた LoRA の重み（adapter）を HF Hub に保存できます。  

---



## 2. 実行手順（最短手順）



### Step 0: Colab の準備
- ランタイムの種類を **GPU** に変更し、GPU が **T4** になっていることを確認してください。
- 過去の実行で環境が壊れている場合は **Runtime > Factory reset** を推奨します。

### Step 1: 依存関係インストール
- 先頭の `pip uninstall` → `pip install` を上から順に実行します。
- 実行後、バージョン表示が想定通りであることを確認します（`unsloth import OK` が出ること）。

### Step 2: Hugging Face へログイン
- `login()` を実行するとトークン入力が求められます。
- 入力するトークンは、**WRITE権限**のものを使用してください。
- ※学習データセットが公開ならログイン無しでも読める場合がありますが、標準手順としてログインします。

### Step 3: 学習の実行
- `main()` が呼ばれ、学習が開始します。
- 学習中に `[LabelStats:train]` が表示されます。これは「loss対象トークンが極端にゼロになっていないか」の健康診断です。

### Step 4: 学習成果物の確認、LoRAアダプタのhuggingfaceへのアップロード
- 学習後、`OUT_LORA_DIR` に以下が保存されます（最低限）：
  - `adapter_config.json`
  - `adapter_model.safetensors`（または `adapter_model.bin`）
  - tokenizer 関連ファイル

---

## 3. 出力（何が生成されるか）



- `OUT_LORA_DIR`（例：`/content/lora_structeval_t_qwen3_4b`）に、
  **LoRAアダプタ（差分重み）**が保存されます。
- このアダプタをベースモデルに適用して推論することで、StructEval-T のスコア改善を狙います。

---



## 4. 学習データセットの説明



### 4.1 データセット概要
本コードで使用するデータセットは以下です：

- HF Dataset: `u-10bei/structured_data_with_cot_dataset_512_v2`  
  https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_512_v2

このデータセットは、**構造化出力（CSV / JSON / XML / TOML / YAML）**を中心とした、
形式変換・抽出タスク向けのSFTデータです。

### 4.2 収録件数・split
- Subset: `default`
- Split: `train`
- 行数：**約 3.65k rows**

### 4.3 カラム（列）構造
Viewer上で確認できる代表的なカラムは以下です。

- `id`（文字列）
- `category`（カテゴリ：複数値）
- `subcategory`（サブカテゴリ：複数値）
- `task`（タスク種別：複数値）
- `seed`（生成や由来を示す識別子）
- `messages`（**OpenAI messages形式のlist**）

特に重要なのが `messages` で、各サンプルは以下のような形です：

```json
[
  {"role": "user", "content": "...指示と入力..."},
  {"role": "assistant", "content": "...期待される出力..."}
]


# 実行コード

## Step 1:依存関係インストール

In [ ]:
# ============================================================
# 0) 依存関係の固定（Colabの“環境ブレ”対策）
# ============================================================
# Colab（無料版）は、ある日突然プリインストール版が変わり、
# それまで動いていた学習コードが壊れることが頻繁にあります。
# そのため、このセルでは「一度全部消す → 互換が確認できたバージョンを入れ直す」
# という“強制的な再現性確保”をしています。
#
# ※Errorが出力されることがありますが、「使用しないライブラリ」に関するエラーであれば、関係なく動作します。

!pip -q uninstall -y numpy pandas datasets trl transformers accelerate peft unsloth unsloth-zoo bitsandbytes xformers
!pip -q install "numpy==2.0.2" "pandas==2.2.2"
!pip -q install \
  "datasets==4.3.0" \
  "trl==0.24.0" \
  "transformers==4.56.2" \
  "accelerate==1.1.0" \
  "peft==0.13.2" \
  "bitsandbytes==0.45.0"
# unsloth / zoo を同系列で揃える（zoo側の要求に合わせる）
# Unsloth本体と unsloth-zoo は“セット運用”が基本です。片方だけ上げると壊れがちです。
!pip -q install "unsloth-zoo==2025.12.7" "unsloth==2025.12.7"



In [ ]:


# ============================================================
# 0.1) バージョン確認（“動くはず”の状態かを目視で確認）
# ============================================================
# ここで想定バージョンとズレている場合、
# 後工程で原因不明のエラーが出る確率が一気に上がります。

import numpy as np, pandas as pd
import datasets, trl, transformers, torch

print("numpy", np.__version__)
print("pandas", pd.__version__)
print("datasets", datasets.__version__)
print("trl", trl.__version__)
print("transformers", transformers.__version__)
print("torch", torch.__version__)

from unsloth import FastLanguageModel
print("unsloth import OK")

# 期待値：
# numpy 2.0.2
# pandas 2.2.2
# datasets 4.3.0（または <4.4.0 で 4.0.* / 4.1.0 以外）
# trl 0.24.0（または 0.18.2〜0.24.0 で 0.19.0以外）
# unsloth import OK


# -----------------------------
# 0) Install (single cell)
# -----------------------------
# NOTE:
# - Colabは初期状態が頻繁に変わるため、ピン留めで安定化します。
#   もし依存関係が壊れている環境であれば、Runtime > Factory reset を推奨。

# このセルを実行して、上の「期待値」にもしなっていない場合は、下記のコメントアウトを外して実行してみてください。
# !pip -q install -U \
#   "numpy==2.0.2" \
#   "pandas==2.2.2" \
#   "datasets==4.3.0" \
#   "trl==0.24.0" \
#   "transformers==4.57.3" \又は、4.56.2
#   "accelerate==1.1.0" \
#   "peft==0.13.2" \
#   "bitsandbytes==0.45.0" \
#   "unsloth-zoo==2025.12.7" \
#   "unsloth==2025.12.7" \
#   "huggingface_hub"


numpy 2.0.2
pandas 2.2.2
datasets 4.3.0
trl 0.24.0
transformers 4.57.3
torch 2.10.0+cu128
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp/ipython-input-1458191610.py:17: UserWarning: WARNING: Unsloth should be imported before [trl, transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!
unsloth import OK


## Step 2: HuggingFace ログイン

Hugging Faceに自分のモデルやデータセットを保存したり、設定を変更したりするには、書き込み用の「トークン」が必要です。
トークンは、以下の手順で取得できます。

ステップ1：設定画面を開く
- Hugging Face にログインします。(https://huggingface.co/)
- 画面右上の自分のアイコンをクリックします。
- メニューの中から 「Settings」（設定）を選択します。

ステップ2：アクセストークンのページへ
- 左側のサイドメニューにある 「Access Tokens」 をクリックします。

ステップ3：新しいトークンを作成する
- 画面中央にある 「+ Create new token」 ボタンをクリックします。
- 設定ウィンドウが開くので、以下の2項目を入力・選択します。
- Token Name: 自分が分かりやすい名前を付けます（例：my-upload-token など）。
- Token type: ここが一番重要です！今回は、学習後のモデル（アダプタ）をアップロードするため、必ず 「Write」 を選択してください。
- 下にある 「Create token」 ボタンを押して完了です。

ステップ4：トークンをコピーして保存する
- 作成されたトークンの横にある コピーアイコン（紙が重なったマーク） をクリックして、トークンをコピーします。

- コピーした文字列は、メモ帳などに貼り付けて大切に保管してください。

⚠️ 大切な注意点
トークンは「パスワード」と同じです： このトークンが他人に知られると、あなたのリポジトリを勝手に書き換えられてしまう恐れがあります。GitHubなどにそのまま貼り付けて公開しないよう、十分注意してください。

In [ ]:

# -----------------------------
# 1) HF login (once)
# -----------------------------
# Hugging Face（HF）はモデルやデータセットをホスティングするサービスです。
# このコードでは「HF Hub上のデータセットを読む」「学習したLoRAをHFにアップする」ためにログインします。
#

from unsloth import FastLanguageModel
import numpy as np, pandas as pd
import datasets, trl, transformers, torch

from huggingface_hub import login, HfApi
login()  # Colab will prompt
api = HfApi()

## Step3:学習の実行

In [ ]:

# ============================================================
# 2) Training code
# ============================================================
# ここからがSFT本体です。
# 大まかな流れ：
#  1) 設定値（モデル名、データセット、LoRA設定、学習率など）を読み込む
#  2) データセットをHFから取得し、必要な形（messages形式）を満たすものだけ残す
#  3) tokenizerで「学習に使うテキスト」を作ってキャッシュする（高速化）
#  4) ベースモデルを4bitでロードし、LoRAアダプタを差し込む
#  5) Trainerで学習を回す
#  6) LoRAアダプタを保存する


- ベースモデル：Qwen3-4B-Instruct-2507
- GPU：T4（無料Colab）でも回るように、メモリ節約を強く意識しています。
- 学習方式：QLoRA（4bitでベースを読み、LoRAアダプタのみ学習）
  - “全部の重み”を学習するのではなく、LoRAアダプタ（軽量差分）だけを学習します。
  - そのため、学習後に保存されるのも「アダプタ」中心になります。

### 使用可能なデータセット
今回、運営において9種類の合成データセットを用意しました。

- 1-1. https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_512_v2
- 1-2. https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_512_v4
- 1-3. https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_512_v5
- 1-4.
https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_512
- 1-5.
https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_v2
- 1-6.
https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset
- 2-1. https://huggingface.co/datasets/daichira/structured-3k-mix-sft
- 2-2. https://huggingface.co/datasets/daichira/structured-5k-mix-sft
- 2-3. https://huggingface.co/datasets/daichira/structured-hard-sft-4k

 この標準コードでは1-1を使用していますが、1-2以降を使用してもOKです。
 - 学習データセットを1-1以外に変更せずとも、後述の環境変数（4.ハイパーパラメータ）を変更することにより、モデル性能が向上する（修了要件を満たす）ことが可能です。
 - さらなる性能向上のため、これらのデータセットに追加で前処理を行ってから学習を行っても差し支えありません。

 注意
- このデータを使用するとスコアが上がることを保証するものではありません．
- ご自身で組み合わせたり，カスタマイズして使用してみてください．
- ただし，詳細資料に記載してあるルールは守ってください．


In [ ]:

import os
import random
import json
import shutil
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple

from datasets import load_dataset, Dataset
from transformers import TrainingArguments, Trainer, TrainerCallback


In [ ]:
# -----------------------------
# 環境変数の設定
# -----------------------------
# 下記の値を書き換えることで、コード本体を編集せずに設定を変更できます。

# 1. モデル・データセット関連
os.environ["SFT_BASE_MODEL"] = "Qwen/Qwen3-4B-Instruct-2507"
os.environ["SFT_DATASET_ID"] = "u-10bei/structured_data_with_cot_dataset_512_v2"
os.environ["SFT_OUT_LORA_DIR"] = "/content/lora_structeval_t_qwen3_4b"

# 2. 学習の基本パラメータ
os.environ["SFT_SEED"] = "3407"
os.environ["SFT_VAL_RATIO"] = "0.05"
os.environ["SFT_MAX_SEQ_LEN"] = "512"

# 3. LoRA (アダプタ) 設定
os.environ["SFT_LORA_R"] = "64"
os.environ["SFT_LORA_ALPHA"] = "128"
os.environ["SFT_LORA_DROPOUT"] = "0"
os.environ["SFT_LORA_TARGET_MODULES"] = "q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj"

# 4. ハイパーパラメータ
os.environ["SFT_EPOCHS"] = "1"
os.environ["SFT_PER_DEVICE_TRAIN_BS"] = "2"
os.environ["SFT_PER_DEVICE_EVAL_BS"] = "2"
os.environ["SFT_GRAD_ACCUM"] = "8"
os.environ["SFT_LR"] = "1e-6"
os.environ["SFT_WARMUP_RATIO"] = "0.1"
os.environ["SFT_WEIGHT_DECAY"] = "0.05"

# 5. ステップ・保存設定
os.environ["SFT_MAX_STEPS"] = "-1" # -1でエポックベース。動作確認時は 10 などに。
os.environ["SFT_LOGGING_STEPS"] = "10"
os.environ["SFT_EVAL_STEPS"] = "50"
os.environ["SFT_SAVE_STEPS"] = "100"
os.environ["SFT_SAVE_TOTAL_LIMIT"] = "2"

# 6. 特殊学習設定 (CoTマスク・アップサンプリング)
os.environ["SFT_MASK_COT"] = "1" # "1" で有効, "0" で無効
os.environ["SFT_OUTPUT_MARKERS"] = "Output:,OUTPUT:,Final:,Answer:,Result:,Response:"
os.environ["SFT_OUTPUT_LEARN_MODE"] = "after_marker" # "after_marker" または "from_marker"
os.environ["SFT_USE_UPSAMPLING"] = "0" # "1" で有効, "0" で無効  # データ2-1,2-2,2-3 専用
os.environ["SFT_UPSAMPLE_RULES"] = '{"xml_to_yaml": 2.0}' # 例: '{"json_to_xml": 1.8, "text_to_yaml": 1.6}' # データ2-1,2-2,2-3 専用

print("環境変数の設定が完了しました。")

環境変数の設定が完了しました。


In [ ]:
# -----------------------------
# 2.1) Config (env-overridable)
# -----------------------------
# “環境変数で上書きできる設定”を用意しています。
# つまり、コードを編集しなくても、Colabの環境変数を変えるだけで
# ベースモデル名、学習率、エポック数などを変更できる設計です。
#
# この設計のメリット：
# - “標準コード”は同じまま、ハイパーパラメータだけ試せる（再現性が高い）

def _getenv(name: str, default: str):
    return os.environ.get(name, default)

def _getenv_int(name: str, default: int) -> int:
    try:
        return int(os.environ.get(name, str(default)))
    except Exception:
        return default

def _getenv_float(name: str, default: float) -> float:
    try:
        return float(os.environ.get(name, str(default)))
    except Exception:
        return default

# 学習の“出発点”となるベースモデル（4B）
BASE_MODEL_ID = _getenv("SFT_BASE_MODEL", "Qwen/Qwen3-4B-Instruct-2507")

# 学習に使うSFTデータセット（HF Hub上に置かれている想定）
DATASET_ID    = _getenv("SFT_DATASET_ID", "u-10bei/structured_data_with_cot_dataset_512_v2")

# 学習後に保存されるLoRAアダプタの出力先（ローカル）
OUT_LORA_DIR  = _getenv("SFT_OUT_LORA_DIR", "/content/lora_structeval_t_qwen3_4b") # HFアップロードするアダプタ名と合わせる

SEED        = _getenv_int("SFT_SEED", 3407)
VAL_RATIO   = _getenv_float("SFT_VAL_RATIO", 0.05)

# 1サンプルあたり最大何トークンまで見るか（長いほど情報を見られるが、GPUメモリと時間が増える）
MAX_SEQ_LEN = _getenv_int("SFT_MAX_SEQ_LEN", 512)

# LoRA Config（＝“どれくらいの表現力を持つ差分を学習するか”）
LORA_R       = _getenv_int("SFT_LORA_R", 64)
LORA_ALPHA   = _getenv_int("SFT_LORA_ALPHA", 128)
LORA_DROPOUT = _getenv_float("SFT_LORA_DROPOUT", 0)
LORA_TARGET_MODULES = (
    _getenv("SFT_LORA_TARGET_MODULES", "q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj").split(",")
)

# Train hyperparams（学習の基本設定）
NUM_TRAIN_EPOCHS            = _getenv_int("SFT_EPOCHS", 1)
PER_DEVICE_TRAIN_BATCH_SIZE = _getenv_int("SFT_PER_DEVICE_TRAIN_BS", 2)
PER_DEVICE_EVAL_BATCH_SIZE  = _getenv_int("SFT_PER_DEVICE_EVAL_BS", 2)

# 勾配累積：GPUに一度に載せられるバッチが小さい時に、複数ステップ分を貯めて“大きいバッチ相当”にする
GRAD_ACCUM                  = _getenv_int("SFT_GRAD_ACCUM", 8)

LR                          = _getenv_float("SFT_LR", 1e-6)
WARMUP_RATIO                = _getenv_float("SFT_WARMUP_RATIO", 0.1)

# Debug / quick check
# MAX_STEPSを小さくすると“動作確認だけ”の短時間学習ができます（本番は -1 のまま）
MAX_STEPS        = _getenv_int("SFT_MAX_STEPS", -1)
LOGGING_STEPS    = _getenv_int("SFT_LOGGING_STEPS", 10)
EVAL_STEPS       = _getenv_int("SFT_EVAL_STEPS", 50)
SAVE_STEPS       = _getenv_int("SFT_SAVE_STEPS", 100)
SAVE_TOTAL_LIMIT = _getenv_int("SFT_SAVE_TOTAL_LIMIT", 2)
WEIGHT_DECAY     = _getenv_float("SFT_WEIGHT_DECAY", 0.05)

# Optional: upsampling rules
# 特定のサブカテゴリ（例：難しいタスク）を“多めに学習させる”ための仕組み。
# 標準ではOFFになっています。
UPSAMPLE_ENABLE     = _getenv("SFT_USE_UPSAMPLING", "0") in ("1","true","True")
UPSAMPLE_RULES_JSON = _getenv("SFT_UPSAMPLE_RULES", "")


# -----------------------------
# 2.2) Seed & Utils
# -----------------------------
# 乱数（シャッフルやサンプリング）を固定して、再現性を担保します。
# seedが同じなら、原則として同じ分割・同じ抽出になりやすいです。

def seed_everything(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(SEED)

def ensure_openai_messages(ds: Dataset, msg_col: str = "messages") -> None:
    # データが「messages: [{role, content}, ...]」形式かをチェックします。
    # これは ChatGPT形式（OpenAIのChat Completions形式に似た）で、
    # tokenizer.apply_chat_template で安全に文字列化するために必要です。
    row0 = ds[0]
    ex = row0.get(msg_col, None)
    if not isinstance(ex, list):
        raise ValueError(f"Dataset must have list-style 'messages'. Got {type(ex)}")

def has_any_nonempty_assistant_turn(msgs: List[Dict[str, Any]]) -> bool:
    # “assistantの発話が空じゃない”ものが1回でも含まれるか？
    # SFTでは「正解例（assistantの出力）」がないと学習できないため。
    return any(m.get("role") == "assistant" and str(m.get("content", "")).strip() != "" for m in msgs)

def ends_with_nonempty_assistant(ex: Dict[str, Any]) -> bool:
    # 最後のターンが assistant の回答になっているサンプルだけを使います。
    # こうしておくと「最後のassistantだけ学習する（assistant-only loss）」設計と相性が良いです。
    msgs = ex.get("messages", [])
    if not msgs or msgs[-1].get("role") != "assistant":
        return False
    c = msgs[-1].get("content", "")
    return isinstance(c, str) and c.strip() != ""

def shuffle_split(ds: Dataset, val_ratio: float, seed: int) -> Tuple[Dataset, Dataset]:
    # データをシャッフルして train/val に分割します。
    # val（検証）を持つことで「学習が進むほど性能が上がっているか／過学習していないか」を見られます。
    ds_shuf = ds.shuffle(seed=seed)
    n = len(ds_shuf)
    n_val = max(1, int(round(n * val_ratio)))
    return ds_shuf.select(range(n_val, n)), ds_shuf.select(range(n_val))

def make_text_cache_builder(tokenizer):
    # messages形式 → 実際にモデルに入力する“1本のテキスト”へ変換する関数を作ります。さらに「トークン長（truncationなし）」もキャッシュします。
    #
    # full_text  : ユーザー＋アシスタント（正解）まで含んだ全文
    # prefix_text: “最後のassistantの直前まで”の文（＝ここからassistantを生成させたい）
    #
    # この2つを持つことで、後のcollatorで「assistant部分だけをloss対象にする境界」を計算できます。

    def _build(batch):
        full_out = []
        prefix_out = []
        full_len_out = []
        prefix_len_out = []

        for msgs in batch["messages"]:
            full = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)
            prefix = tokenizer.apply_chat_template(msgs[:-1], tokenize=False, add_generation_prompt=True)

            full_out.append(full)
            prefix_out.append(prefix)

            # 重要：ここで truncation=False で token 長だけ計算してキャッシュする
            # add_special_tokens=False はあなたの現行設計に合わせる（テンプレ側で必要トークンが入る想定）
            full_ids = tokenizer(full, add_special_tokens=False, truncation=False)["input_ids"]
            prefix_ids = tokenizer(prefix, add_special_tokens=False, truncation=False)["input_ids"]

            full_len_out.append(len(full_ids))
            prefix_len_out.append(len(prefix_ids))

        return {
            "full_text": full_out,
            "prefix_text": prefix_out,
            "full_input_ids_len": full_len_out,
            "prefix_input_ids_len": prefix_len_out,
        }

    return _build



# -----------------------------
# 2.3) Collator (assistant-only loss)
# -----------------------------
# collatorは「生のサンプル群 → 学習に必要なテンソル(input_ids/labels等)」に変換する部品です。
#
# ここがこの学習コードの“設計思想”の核心：
# - 入力（user/system）も含めてモデルには読ませる
# - ただし loss（誤差）を計算するのは assistant の出力部分だけ
#
# これにより：
# - 「プロンプトを丸暗記させる」方向に学習が引っ張られにくい
# - “回答の形式”や“出力の正確さ”に学習の力点を置きます。

# 使用データセットによる仕様の違い
# データセット1：Output: が 100% なので CoT マスクが常に動き、Output本体だけ学習
# データセット2：Output: 系ラベルが存在しないため、CoTマスクは発動せず、“出力本体”を学習

# --- CoT mask settings (env overridable) ---
MASK_COT = _getenv("SFT_MASK_COT", "1") in ("1","true","True")
OUTPUT_MARKERS = [s.strip() for s in _getenv(
    "SFT_OUTPUT_MARKERS",
    "Output:,OUTPUT:,Final:,Answer:,Result:,Response:"
).split(",") if s.strip()]
OUTPUT_LEARN_MODE = _getenv("SFT_OUTPUT_LEARN_MODE", "after_marker")  # after_marker / from_marker

@dataclass
class AssistantOnlyCollatorCached:
    tokenizer: Any
    max_length: int = MAX_SEQ_LEN

    def _find_subsequence(self, seq: List[int], sub: List[int]) -> int:
        if not sub or len(sub) > len(seq):
            return -1
        for i in range(0, len(seq) - len(sub) + 1):
            if seq[i:i+len(sub)] == sub:
                return i
        return -1

    def __call__(self, batch: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        tok = self.tokenizer
        full_texts   = [ex["full_text"] for ex in batch]
        prefix_texts = [ex["prefix_text"] for ex in batch]

        old_trunc = getattr(tok, "truncation_side", "right")
        old_pad   = getattr(tok, "padding_side", "right")
        tok.truncation_side = "left"
        tok.padding_side    = "right"

        try:
            full_enc_tr = tok(
                full_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=self.max_length,
                add_special_tokens=False,
            )
            input_ids = full_enc_tr["input_ids"]
            attention_mask = full_enc_tr["attention_mask"]
            labels = torch.full_like(input_ids, fill_value=-100)

            full_ids_nt   = tok(full_texts,   return_tensors=None, padding=False, truncation=False, add_special_tokens=False)["input_ids"]
            prefix_ids_nt = tok(prefix_texts, return_tensors=None, padding=False, truncation=False, add_special_tokens=False)["input_ids"]

            marker_token_seqs = []
            if MASK_COT and OUTPUT_MARKERS:
                for m in OUTPUT_MARKERS:
                    mid = tok(m, add_special_tokens=False, truncation=False)["input_ids"]
                    if not mid:
                        continue
                    mid_nl = tok(m + "\n", add_special_tokens=False, truncation=False)["input_ids"]
                    mid_crlf = tok(m + "\r\n", add_special_tokens=False, truncation=False)["input_ids"]
                    marker_token_seqs.append((mid, mid_nl, mid_crlf))

            for i in range(input_ids.size(0)):
                trunc_left = max(0, len(full_ids_nt[i]) - self.max_length)
                boundary = len(prefix_ids_nt[i]) - trunc_left
                full_len_tr = int(attention_mask[i].sum().item())

                # assistant開始が見えていない => 学習対象外（元コード方針を維持）
                if boundary <= 0 or boundary >= full_len_tr:
                    continue

                span_start = boundary
                span_end   = full_len_tr

                # デフォルト：assistant全体を学習（データセット2はここに落ちる）
                learn_start = span_start

                # CoTマスク：Output marker が見つかったときだけ学習開始点を進める（データセット1で発動）
                if MASK_COT and marker_token_seqs:
                    visible_ids = input_ids[i, :full_len_tr].tolist()
                    assistant_ids = visible_ids[span_start:span_end]

                    best_out = None  # (out_pos, after_pos)
                    for mid, mid_nl, mid_crlf in marker_token_seqs:
                        # 改行付き優先
                        p = self._find_subsequence(assistant_ids, mid_nl)
                        if p != -1:
                            out_pos = span_start + p
                            after_pos = out_pos + len(mid_nl)
                        else:
                            p = self._find_subsequence(assistant_ids, mid_crlf)
                            if p != -1:
                                out_pos = span_start + p
                                after_pos = out_pos + len(mid_crlf)
                            else:
                                p = self._find_subsequence(assistant_ids, mid)
                                if p == -1:
                                    continue
                                out_pos = span_start + p
                                after_pos = out_pos + len(mid)

                        if (best_out is None) or (out_pos < best_out[0]):
                            best_out = (out_pos, after_pos)

                    if best_out is not None:
                        out_pos, after_pos = best_out
                        if OUTPUT_LEARN_MODE == "from_marker":
                            learn_start = out_pos
                        else:
                            learn_start = after_pos
                        learn_start = max(span_start, min(learn_start, span_end))

                if learn_start < span_end:
                    labels[i, learn_start:span_end] = input_ids[i, learn_start:span_end]

            labels[attention_mask == 0] = -100
            return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
        finally:
            tok.truncation_side = old_trunc
            tok.padding_side    = old_pad

import random, torch

@torch.no_grad()
def filter_has_supervision(ds, collator):
    keep = []
    for i in range(len(ds)):
        out = collator([ds[i]])
        if (out["labels"][0] != -100).sum().item() > 0:
            keep.append(i)
    return ds.select(keep)


def count_all_masked(ds, collator, n=200, seed=3407):
    rng = random.Random(seed)
    n = min(n, len(ds))
    idxs = [rng.randrange(0, len(ds)) for _ in range(n)]
    all_masked = 0
    for i in idxs:
        out = collator([ds[i]])
        labels = out["labels"][0]
        if (labels != -100).sum().item() == 0:
            all_masked += 1
    print(f"[CHECK] all-masked samples in {n}: {all_masked} ({all_masked/max(1,n):.1%})")


# -----------------------------
# 2.4) Optional upsampling
# -----------------------------
# upsamplingは「特定の種類のデータを多めに学習させる」テクニックです。
# 例：
# - JSONは得意だがYAMLは苦手 → YAML関連サンプルを2倍にする
# - 特定のsubcategoryが点数に効く → そこを厚くする
# ただし、やりすぎると他が弱くなることもあります（トレードオフ）。
# 学習データセットの品質が悪い等の原因で、却って性能が低下することもあります。
# その場合、学習データセットを観察し、追加の前処理が有効であることも多いです。

def apply_upsampling(train_ds: Dataset) -> Dataset:
    if not UPSAMPLE_ENABLE or not UPSAMPLE_RULES_JSON:
        return train_ds
    try:
        rules = json.loads(UPSAMPLE_RULES_JSON)
        if not isinstance(rules, dict) or not rules:
            return train_ds
    except Exception:
        return train_ds

    packs = train_ds["subcategory"] if "subcategory" in train_ds.column_names else [None]*len(train_ds)
    pack_field = train_ds["pack"] if "pack" in train_ds.column_names else [None]*len(train_ds)

    w = []
    for sub, pk in zip(packs, pack_field):
        weight = 1.0
        ssub = str(sub or "")
        spk  = str(pk or "")
        for pat, mult in rules.items():
            try:
                m = float(mult)
            except Exception:
                m = 1.0
            if pat.startswith("pack:"):
                if spk == pat.split(":",1)[1]:
                    weight *= max(0.0, m)
            else:
                if pat in ssub:
                    weight *= max(0.0, m)
        w.append(weight)

    w = np.asarray(w, dtype=np.float64)
    if (w <= 0).all() or w.sum() == 0:
        return train_ds

    p = w / w.sum()
    n = len(train_ds)
    idx = np.random.choice(np.arange(n), size=n, replace=True, p=p)
    print("[UPSAMPLE] rules:", rules)
    return train_ds.select(idx.tolist())


# -----------------------------
# 2.5) Callback (monitor)
# -----------------------------
# 学習中のデバッグ用コールバックです。
# ここでは「labelsのうち、実際にloss対象になっているトークン割合」を時々表示します。
#
# 意味：
# - valid_ratio が極端に小さい → “学習していない”のと同じ（ラベルがほぼ -100）
# - valid_ratio が適度にある → assistant部分にしっかりlossが乗っている
#
# 初学者向けに言うと：
# - これは“学習がちゃんと効いているかの健康診断”です。

class LabelStatsCallback(TrainerCallback):
    def __init__(self, dataset, collator, name="train", every_n_steps=100):
        self.dataset, self.collator, self.name, self.every_n_steps = dataset, collator, name, every_n_steps

    @torch.no_grad()
    def on_step_end(self, args, state, control, **kwargs):
        if (state.global_step % self.every_n_steps) == 0:
            batch = [self.dataset[random.randint(0, len(self.dataset)-1)] for _ in range(8)]
            out = self.collator(batch)
            valid = (out["labels"] != -100).sum().item()
            total = (out["attention_mask"] == 1).sum().item()
            print(f"\n[LabelStats:{self.name}] step={state.global_step} valid_ratio={valid/max(1,total):.4f}")


# -----------------------------
# 2.6) Main
# -----------------------------
# 学習を実行します。

def main():
    os.makedirs(OUT_LORA_DIR, exist_ok=True)

    # if you used /content/your_id cache dirs etc, remove to avoid confusion
    if os.path.exists("/content/your_id"):
        shutil.rmtree("/content/your_id")

    print(f"[INFO] Loading dataset from HF Hub: {DATASET_ID}")
    ds_all = load_dataset(DATASET_ID, split="train")

    # データ形式チェック（messagesがlistであること）
    ensure_openai_messages(ds_all)

    # 学習できるサンプルだけ残す（assistantが空なら教師信号が無い）
    ds_all = ds_all.filter(lambda ex: has_any_nonempty_assistant_turn(ex["messages"]))
    ds_all = ds_all.filter(ends_with_nonempty_assistant)

    # train/val分割
    train_ds, val_ds = shuffle_split(ds_all, VAL_RATIO, SEED)

    # Optional: upsampling by rule（分割後に適用）
    train_ds = apply_upsampling(train_ds)

    print("[INFO] Loading base model:", BASE_MODEL_ID)

    # Unslothでベースモデルを読み込む（4bitロードで省メモリ）
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_ID,
        max_seq_length=MAX_SEQ_LEN,
        dtype=None,
        load_in_4bit=True,
    )

    # Cache chat template renders（tokenizerが必要なのでここで初めてbuild_cacheを作る）
    build_cache = make_text_cache_builder(tokenizer)

    train_ds = train_ds.map(build_cache, batched=True, num_proc=1, desc="Caching train")
    val_ds   = val_ds.map(build_cache,   batched=True, num_proc=1, desc="Caching val")

    # Attach LoRA
    # ここで「学習される部分（LoRAアダプタ）」をモデルに追加します。
    # 学習対象は LoRA のパラメータだけになり、ベースモデルの巨大な重みは固定されます。
    model = FastLanguageModel.get_peft_model(
        model,
        r=LORA_R,
        target_modules=LORA_TARGET_MODULES,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        use_gradient_checkpointing="unsloth",
        random_state=SEED,
    )


    # Transformersの引数名がバージョンで揺れることがあります。
    # 今回のバージョンでは eval_strategy を使います。
    args = TrainingArguments(
        output_dir=OUT_LORA_DIR,
        num_train_epochs=NUM_TRAIN_EPOCHS,
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        learning_rate=LR,
        warmup_ratio=WARMUP_RATIO,
        lr_scheduler_type="cosine",
        weight_decay=WEIGHT_DECAY,

        logging_steps=LOGGING_STEPS,

        eval_strategy="steps",
        eval_steps=EVAL_STEPS,

        save_strategy="steps",
        save_steps=SAVE_STEPS,
        save_total_limit=SAVE_TOTAL_LIMIT,

        max_steps=MAX_STEPS,  # -1 => epoch-based

        bf16=False,
        fp16=True,            # T4向け（T4はbf16が弱いのでfp16を使うのが一般的）

        push_to_hub=False,
        report_to="none",

        group_by_length=False,
        remove_unused_columns=False,
    )

    # assistant-only loss の collator を使う
    collator = AssistantOnlyCollatorCached(tokenizer=tokenizer, max_length=MAX_SEQ_LEN)

    # --- NaN対策：all-masked（教師トークン0）を除去して評価を安定化 ---
    print("[INFO] Checking all-masked samples before filtering...")
    count_all_masked(val_ds, collator, n=len(val_ds), seed=SEED)

    print("[INFO] Filtering train/val to remove all-masked samples...")
    train_ds = filter_has_supervision(train_ds, collator)
    val_ds   = filter_has_supervision(val_ds, collator)

    print("[INFO] New sizes:", "train =", len(train_ds), "val =", len(val_ds))
    print("[INFO] Checking all-masked samples after filtering...")
    count_all_masked(val_ds, collator, n=len(val_ds), seed=SEED)


    # Trainer（Transformersの標準学習ループ）
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collator,
        tokenizer=tokenizer,
    )

    # 監視用コールバックを追加（学習が効いているかのヘルスチェック）
    trainer.add_callback(LabelStatsCallback(train_ds, collator, name="train", every_n_steps=LOGGING_STEPS))

    print("[INFO] Starting training...")
    trainer.train()

    # 学習後の保存：LoRAアダプタ＆tokenizer
    print("[INFO] Saving adapter & tokenizer...")
    model.save_pretrained(OUT_LORA_DIR)
    tokenizer.save_pretrained(OUT_LORA_DIR)
    print(f"[INFO] Done. Saved to {OUT_LORA_DIR}")

if __name__ == "__main__":
    main()



## Step 4: 学習成果物の確認と、LoRAアダプタのhuggingfaceへのアップロード
学習後、OUT_LORA_DIR に以下が保存されます（最低限）ので、確認してください。
- adapter_config.json
- adapter_model.safetensors（または adapter_model.bin）
- tokenizer 関連ファイル
<br>

下記に従って"README.md"を記載してから、HuggingFaceにアダプタアップロードを実行してください。



### ① README.md の正しい書き方

#### README.md の役割（最重要）

Hugging Face では **README.md = モデルカード**です。
「このLoRAは何を学習し、どう使い、何に注意すべきか」を **第三者が再利用できる水準で説明する義務**があります。

README が不十分なモデルは、

* OSSとして不適切
* 学習内容が不透明
* ライセンス違反リスクあり
  と評価されます。

---

#### 必須構成（この順で書くこと）

##### 1. YAMLメタデータ（必須）

```yaml
---
base_model: Qwen/Qwen3-4B-Instruct-2507
datasets:
- u-10bei/structured_data_with_cot_dataset_512_v2
language:
- en
license: Apache-2.0
library_name: peft
pipeline_tag: text-generation
tags:
- qlora
- lora
- structured-output
---
```

**理由**

* HF検索・分類・再現性に必須
* 無いと「壊れたモデルカード」扱いになる

---

##### 2. モデル概要（What）

```md
# qwen3-4b-structured-output-lora

This repository provides a **LoRA adapter** fine-tuned from
**Qwen3-4B-Instruct-2507** using **QLoRA (4-bit, Unsloth)**.

This repository contains **LoRA adapter weights only**.
The base model must be loaded separately.
```

**必須ポイント**

* 「LoRAアダプタのみ」であることを明記
* ベースモデル名を明示

---

##### 3. 学習目的・設計思想（Why）

```md
## Training Objective

This adapter is trained to improve **structured output accuracy**
(JSON / YAML / XML / TOML / CSV).

Loss is applied only to the final assistant output,
while intermediate reasoning (Chain-of-Thought) is masked.
```

**今回の講座では特に重要**

* assistant-only loss
* CoT mask（Output: 以降のみ学習）

---

##### 4. 学習設定（How）

```md
## Training Configuration

- Base model: Qwen3-4B-Instruct-2507
- Method: QLoRA (4-bit)
- Max sequence length: 512
- Epochs: 1
- Learning rate: 1e-6
- LoRA: r=64, alpha=128
```

**再現性のため必須**

---

##### 5. 使用方法（How to use）

````md
## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base = "Qwen/Qwen3-4B-Instruct-2507"
adapter = "your_id/your-repo"

tokenizer = AutoTokenizer.from_pretrained(base)
model = AutoModelForCausalLM.from_pretrained(
    base,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, adapter)
````

````

---

##### 6. データセット・ライセンス注意（必須・重要）
```md
## Sources & Terms (IMPORTANT)

Training data: u-10bei/structured_data_with_cot_dataset_512_v2

Dataset License: MIT License. This dataset is used and distributed under the terms of the MIT License.
Compliance: Users must comply with the MIT license (including copyright notice) and the base model's original terms of use.
````
---
##### 実行コードの見本

- 【課題】最低限、モデルタイトルの欄は、必ず自身で書き込んで下さい。
- 使用データセットを変更した場合には、"Dataset License","Compliance" 欄も適切な形に書き換えてください。

In [ ]:
# -----------------------------
# README.md（モデルカード）を OUT_LORA_DIR に生成
# -----------------------------
# 学習完了後に実行し、Hugging Face の README.md（モデルカード）を生成
# ベースモデル名・データセット名・学習ハイパーパラメータはコードの変数から自動同期

import os

os.makedirs(OUT_LORA_DIR, exist_ok=True)

# ------------------------------------------------------------------
# 補助関数の定義
# ------------------------------------------------------------------
def _s(x, default=""):
    try:
        v = str(x)
        return v if v.strip() else default
    except Exception:
        return default

def _fmt_lr(x) -> str:
    """
    Learning Rate の表記を整えるための関数。

    - 数値として解釈できる場合：
      指数表記（例: 1e-6）に整形する
    - 数値として解釈できない場合：
      元の値をそのまま文字列として出力する
      （誤った値を生成しないための安全策）
    """
    try:
        return f"{float(x):.0e}"
    except Exception:
        return _s(x, "")


# ------------------------------------------------------------------
# 学習コードの変数から値を取得（README と自動同期）
# ------------------------------------------------------------------
base_model_id = _s(BASE_MODEL_ID, "Qwen/Qwen3-4B-Instruct-2507")
dataset_id = _s(DATASET_ID, "https://huggingface.co/datasets/u-10bei/structured_data_with_cot_dataset_512_v2")

max_seq_len = int(MAX_SEQ_LEN)
epochs = int(NUM_TRAIN_EPOCHS)
lr_str = _fmt_lr(LR)

lora_r = int(LORA_R)
lora_alpha = int(LORA_ALPHA)

# NOTE:
# - YAML front matter の license は
#   「この LoRA アダプタ（リポジトリ）のライセンス表明」を意味する。
# - 必要に応じて環境変数で差し替え可能。
repo_license = os.environ.get("SFT_REPO_LICENSE", "apache-2.0")

# README 内に記載するモデルタイトル
# 変更したい場合は README.md を手書きで調整
title_line = "＜【課題】ここは自分で記入して下さい＞" #例： qwen3-4b-structured-output-lora

# ------------------------------------------------------------------
# README.md 本文の生成
# （説明テキストに準拠し、変数部分のみを自動置換）
# ------------------------------------------------------------------
readme_md = f"""---
base_model: {base_model_id}
datasets:
- {dataset_id}
language:
- en
license: {repo_license}
library_name: peft
pipeline_tag: text-generation
tags:
- qlora
- lora
- structured-output
---

{title_line}

This repository provides a **LoRA adapter** fine-tuned from
**{base_model_id}** using **QLoRA (4-bit, Unsloth)**.

This repository contains **LoRA adapter weights only**.
The base model must be loaded separately.

## Training Objective

This adapter is trained to improve **structured output accuracy**
(JSON / YAML / XML / TOML / CSV).

Loss is applied only to the final assistant output,
while intermediate reasoning (Chain-of-Thought) is masked.

## Training Configuration

- Base model: {base_model_id}
- Method: QLoRA (4-bit)
- Max sequence length: {max_seq_len}
- Epochs: {epochs}
- Learning rate: {lr_str}
- LoRA: r={lora_r}, alpha={lora_alpha}

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base = "{base_model_id}"
adapter = "your_id/your-repo"

tokenizer = AutoTokenizer.from_pretrained(base)
model = AutoModelForCausalLM.from_pretrained(
    base,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, adapter)
```

## Sources & Terms (IMPORTANT)

Training data: {dataset_id}

Dataset License: MIT License. This dataset is used and distributed under the terms of the MIT License.
Compliance: Users must comply with the MIT license (including copyright notice) and the base model's original terms of use.
"""
# ------------------------------------------------------------------
# README.md の書き込み
# ------------------------------------------------------------------

readme_path = os.path.join(OUT_LORA_DIR, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_md)

# ------------------------------------------------------------------
# 動作確認
# ------------------------------------------------------------------

assert os.path.exists(readme_path), "README.md was not written."
assert readme_md.lstrip().startswith("---\n"), (
    "README.md must start with YAML front matter."
)
# 修正: 先頭の --- は改行なしで始まるため count("\n---\n") には含まれない。
# そのため、閉じタグの分として 1回以上あればOKとする。
assert readme_md.count("\n---\n") >= 1, (
    "YAML front matter must be closed properly."
)

print(f"[INFO] README.md written to: {readme_path}")
print("[INFO] Preview (first 30 lines):")
for i, line in enumerate(readme_md.splitlines()[:30], start=1):
    print(f"{i:02d}: {line}")

---

### ② README.md の HF アップロードコード

以下は **README.md を自動生成しません**。

* 直前のコードを参照してREADME.md 完成させ、 OUT_LORA_DIR に保存してから実行してください。
* アップロード対象として README.md を必須化
* README.md が存在しない場合 → **エラー**
---

In [ ]:
# ============================================================
# 3) LoRAアダプターをHugging Faceへアップロード (作成済みのREADMEを含む)
# ============================================================

import fnmatch
import shutil
from pathlib import Path
from huggingface_hub import HfApi

# Hugging Face APIの操作用インスタンスを作成
api = HfApi()

# 各種パスや設定の準備
LORA_SAVE_DIR = Path(OUT_LORA_DIR)  # 学習済みモデルが保存されているディレクトリ
HF_REPO_ID    = _getenv("HF_REPO_ID", "your_id/your-lora-repo")  # アップロード先のレポジトリID

# 非公開設定の確認（環境変数が '1' または 'true' ならプライベート設定にする）
PRIVATE       = _getenv("HF_PRIVATE", "1") in ("1","true","True")

# -----------------------------
# 3.1) 必須ファイルの存在確認
# -----------------------------
# アップロードに最低限必要なファイルを定義します
required_files = {
    "adapter_config.json", # LoRAの設定ファイル
    "README.md",           # 受講生が作成した解説文書
}

# 保存ディレクトリにあるファイル名のリストを取得
present = {p.name for p in LORA_SAVE_DIR.iterdir() if p.is_file()}

# 足りないファイルをリストアップ
missing = [f for f in required_files if f not in present]

# モデル本体（adapter_model.safetensors または .bin）が存在するか確認
if not any(f.startswith("adapter_model.") for f in present):
    missing.append("adapter_model.(safetensors|bin)")

# 必須ファイルが欠けている場合は、エラーを表示して処理を中断します
if missing:
    raise RuntimeError(
        "アップロードを中止しました。\n"
        "以下の必須ファイルが見つかりません:\n"
        + "\n".join(f"- {m}" for m in missing) +
        "\n\nアップロード前に、README.md を手書きで作成し保存してください。"
    )

print("✅ 必須ファイルの確認が完了しました。")

# -----------------------------
# 3.2) アップロード対象の選別（ホワイトリスト）
# -----------------------------
# 不要な一時ファイルなどをアップロードしないよう、許可するファイル形式を指定します
ALLOW_PATTERNS = [
    "README.md",
    "adapter_config.json",
    "adapter_model.*",
    "tokenizer.*",
    "special_tokens_map.json",
    "*.json",
]

def is_allowed(name: str) -> bool:
    """ファイル名が許可パターンに一致するか判定する関数"""
    return any(fnmatch.fnmatch(name, pat) for pat in ALLOW_PATTERNS)

# アップロード用の一時フォルダ（ステージング領域）を作成
STAGE_DIR = Path("/content/hf_upload_stage")

if STAGE_DIR.exists():
    shutil.rmtree(STAGE_DIR) # 既存のフォルダがあれば一旦削除
STAGE_DIR.mkdir(parents=True)

# 許可されたファイルだけを一時フォルダにコピー
for p in LORA_SAVE_DIR.iterdir():
    if p.is_file() and is_allowed(p.name):
        (STAGE_DIR / p.name).write_bytes(p.read_bytes())

print("📦 アップロード対象ファイル:", [p.name for p in STAGE_DIR.iterdir()])

# -----------------------------
# 3.3) リポジトリ作成とアップロード
# -----------------------------

# Hugging Face上にリポジトリを作成（既に存在していてもOK）
api.create_repo(
    repo_id=HF_REPO_ID,
    repo_type="model",
    exist_ok=True,
    private=PRIVATE,
)

# 一時フォルダの内容をまるごとアップロード
api.upload_folder(
    folder_path=str(STAGE_DIR),
    repo_id=HF_REPO_ID,
    repo_type="model",
    commit_message="Upload LoRA adapter (README written by author)",
)

print("✅ アップロードが正常に完了しました。")
print(f"URL: https://huggingface.co/{HF_REPO_ID}")